# Quantitative risk models


## Summary
Vega can use different risk models as for different underlying assets and different derivatives one has to choose an appropriate model. Every quantitative risk model used by Vega needs to be able to provide the following.

1. Risk factors, currently for futures but for other products as Vega develops. The risk factor *linearizes* the risk measure used for margin requirements. See `0019-MCAL-margin_calculator.md` for details of how these are used.
1. Given a probability level $\alpha \in (0.9,1)$, time horizon $\tau > 0$ and current price $S$ return the $\Delta^-$ and $\Delta^+$ such that 
$$
\mathbb P(S-\Delta^- \leq S_\tau \leq S+\Delta^+) \geq 1-\alpha\,.
$$
See the price monitoring spec for details of how these are used.
1. Given current price $S$, time horizon $\tau > 0$, a list of price levels $S < S_1 < S_2 < \cdots < S_N^+$ return the probability of trading at each of these price levels defined below. See "Probability weighted liquidity measure" spec for how these are used.
1. Calibration outputs (not required for first release of Mainnet, model parameters result from governance vote per market).

The “quant risk suite” with Vega consists a _quantitative risk model_, _margin calculator_ and _calibrator_.

A market parameter specifies which _quantitative risk model_ is used for a given risk universe (market, see Section 3.5 of the Vega white paper).


## Reference-level explanation

### _Quantitative risk model_
The relevant quantitative risk model for a market is specified on a tradeable instrument. The role of the quantitative risk model is to calculate **risk factors** which are used in the **_margin calculator_** (see below)
as well as probabilities of various price moves used for price monitoring and measuring liquidity. 

To achieve this it utilises the quantitative maths library.

The quantitative risk model may take one or more of the following as inputs:
* risk parameters (e.g. volatility)
* product parameters (e.g. minimum contract size)
* order book data (full current order book with volume aggregated at price levels)
* position data (for each trader)
* event data (e.g. passage of time) (Not for Futures / Nicenet )

The quantitative risk model returns two risk factors:

1. Rounded Long position risk factor
1. Rounded Short position risk factor

The quantitative risk model is able to utilise a relevant method from the quant math library to perform the calculations.

The quant math library calculates:
1. Long position risk factor
1. Short position risk factor
1. Guaranteed accuracy (applicable to both risk factors)
1. The max move up/down ($\Delta^-$ and $\Delta^+$ defined in the [Summary](#Summary)) given current price level and a projection horizon such that the resulting prices are within a specified probability level.
1. Probability of trading at a given price level (see below).

#### Calculating the probability of trading
This is well defined only if we have best bid $S_{\text{best bid}}$ and best ask $S_{\text{best ask}}$ on the order book. 
The rest of information comes from the quant risk model that has the probability density of price distribution at time horizon $\tau > 0$ i.e. we have 
$$
f(x;S) = \mathbb P(S_\tau \in [x, x+dx) | S)\,, 
$$
where $S$ is current price level and $S_\tau$ is a future price, after time $\tau > 0$.

Given a price level $S$, time horizon $\tau > 0$, a list of price levels $S < S_1 < S_2 < \cdots < S_{N^+}$ or a list of price levels $S > S_1 > S_2 > \cdots > S_{N^-}$ return the probability of trading $p(S_i)$ at each of these as defined below. 

1. If $S_i \in [S_{\text{best bid}}, S_{\text{best ask}}]$ for $i=1,\ldots,N^+$ or $i=1,\ldots,N^-$ return $p(S_i) = 1$. 
That is, between best bid and best ask the probability of trading is one.  
2. If $S_i > S_{\text{best ask}}$ for $i=1,\ldots,N^+$ then return 
$$
p(S_i) = \int_{S_i}^{S_{max}} f(x;S_{\text{best ask}})\,dx
$$
3. If $S_i < S_{\text{best bid}}$ for $ =1,\ldots,N^-$ then return
$$
p(S_i) = \int_{S_{min}}^{S_i} f(x;S_{\text{best bid}})\,dx\,.
$$

#### When to not update risk factors

The call to the quantitative math library should *only* be made if any of the above inputs have changed from last time; if no input has changed then the quantitative risk model doesn't need to update the risk factors.  

#### When to update risk factors

Risk factors are an input to the [margin calculation](./0019-MCAL-margin_calculators.md) and are calculated using a quantitative risk model.

Risk factors are updated if  
* An update risk factors call is not already in progress asynchronously; AND
* Any of the required inputs to risk factors change. Examples 1. when the calibrator has updated any risk parameters. 2. a specified period of time has elapsed (period can = 0 for always recalculate) for re-calculating risk factors. This period of time is defined as a risk parameter (see [market framework](./0001-MKTF-market_framework.md)).

Risk factors are also updated if on creation of a new market that does not yet have risk factors, as any active market needs to have risk factors.

#### Risk factors and consensus

All new risk factors will need to be agreed via consensus when either (or both): 
- asynchronous updates
- the other cause will be due to floating point non-determinism

The rounding should remove all digits beyond the guaranteed accuracy. 

Example: If `Long position risk factor = 1.23456789` and `Guaranteed accuracy = 0.001` then 
`Rounded Long position risk factor = 1.234`. 



## Acceptance Criteria

1. Different markets can have a different risk model (i.e. a market A can be specified to run with risk model R1 while market B can be specified to run with risk model R2). (<a name="0018-RSKM-001" href="#0018-RSKM-001">0018-RSKM-001</a>)
1. If any of the input data has changed then an update to risk factors is initiated. (<a name="0018-RSKM-003" href="#0018-RSKM-003">0018-RSKM-003</a>)
1. Risk factors are agreed upon by consensus. (<a name="0018-RSKM-004" href="#0018-RSKM-004">0018-RSKM-004</a>)
1. If the risk factor calculation reports "guaranteed accuracy" then the risk factors are appropriately rounded. (<a name="0018-RSKM-005" href="#0018-RSKM-005">0018-RSKM-005</a>)
1. Quant risk suite can compute max move up/down ($\Delta^-$ and $\Delta^+$) given current price level and a projection horizon such that the resulting prices are within a specified probability level. (<a name="0018-RSKM-007" href="#0018-RSKM-007">0018-RSKM-007</a>)
1. Quant risk suite can compute probability of trading at a given level. (<a name="0018-RSKM-008" href="#0018-RSKM-008">0018-RSKM-008</a>)
1. Lognormal risk model has defined ranges of valid parameters and market proposals and market update proposals are checked against these. The ranges can be found in core (https://github.com/vegaprotocol/vega/blob/develop/commands/proposal_submission.go#L820). (<a name="0018-RSKM-009" href="#0018-RSKM-009">0018-RSKM-009</a>)